# Description

Calling average and standard deviation functions/

In [13]:
import pandas as pd
import numpy as np

In [2]:
from openai import OpenAI
client = OpenAI()

In [3]:
type(np.array([1,2,3]))

numpy.ndarray

In [4]:
assistant = client.beta.assistants.create(
  instructions="You are a financial quantitative portfolio manager assistant",
  model="gpt-4-turbo",
  tools=[
    {
      "type": "function",
      "function": {
        "name": "mean",
        "description": "Calculate arithmetic mean",
        "parameters": {
          "type": "object",
          "properties": {
            "data": {
              "type": "array",
              "items": {
                "type": "number"
              }
            }
          },
          "required": ["data"]
        }
      }
    },
    {
      "type": "function",
      "function": {
        "name": "std",
        "description": "Calculate standard deviation",
        "parameters": {
          "type": "object",
          "properties": {
            "data": {
              "type": "array",
              "items": {
                "type": "number"
              }
            }
          },
          "required": ["data"]
        }
      }
    }
  ]
)


### Creating a thread

Give instructions to the GPT

In [5]:
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="""Calculate both the mean and standard deviation of this series of numbers:
              
            [1,2,3,4,5,6,7,8,9,10]
    """,
)

### Run

In [6]:
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

### Analyse output

So now the API was called and the model run.

Given we passed some tools, the GPT will try to call the right functions - if necessary.

If tools are called then **run.status == requires_action**, this will tell us tools need to be used.

GPT won't allow the code to run auotmatically and we have to pipe the GPT output into the code to run the functions we want to be called.

This is an example and the functions simply returns 57 degrees, and 6% chance of rain

In [7]:
if run.status == 'completed':
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run.status)
 
# Define the list to store tool outputs
tool_outputs = []

requires_action


**And these are the functions that GPT wants to call and the relative parameters**

In [8]:
run.required_action.submit_tool_outputs.tool_calls

[RequiredActionFunctionToolCall(id='call_MKaxUZPn88rCbqLrgsHn0l3T', function=Function(arguments='{"data": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}', name='mean'), type='function'),
 RequiredActionFunctionToolCall(id='call_ou5qX7q9xZckfak3LWGzrqBn', function=Function(arguments='{"data": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}', name='std'), type='function')]

### Tool output

We create a list **tool_output**, where we sget the function output and the **tool_call_id**.

the tool id is necessary because we will feed back to the GPT the functions output, but we have to let the GPT knows to which function the output is referring to.

The GPT will then be able to provide the correct answer, using the tools you gave it.

In [9]:
import numpy as np

tool_outputs = []

# Loop through each tool in the required action section
for tool in run.required_action.submit_tool_outputs.tool_calls:
    arguments = json.loads(tool.function.arguments)
    
    if tool.function.name == "mean":
        # Retrieve data from tool parameters
        data = arguments['data']
        # Compute mean and append to tool_outputs
        output_value = str(np.mean(data))
        tool_outputs.append({
            "tool_call_id": tool.id,
            "output": output_value
        })
    elif tool.function.name == "std":
        # Retrieve data from tool parameters
        data = arguments['data']
        # Compute standard deviation and append to tool_outputs
        output_value = str(np.std(data))
        tool_outputs.append({
            "tool_call_id": tool.id,
            "output": output_value
        })

if tool_outputs:
    try:
        run = client.beta.threads.runs.submit_tool_outputs_and_poll(
            thread_id=thread.id,
            run_id=run.id,
            tool_outputs=tool_outputs
        )
        print("Tool outputs submitted successfully.")
    except Exception as e:
        print("Failed to submit tool outputs:", e)
else:
    print("No tool outputs to submit.")


Tool outputs submitted successfully.


In [11]:
if run.status == 'completed':
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run.status)

SyncCursorPage[Message](data=[Message(id='msg_ONT98fi81ERWZK8hkc2KmGJD', assistant_id='asst_O3w8tazAUeqOy042e2eTfeW3', completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='The mean of the series is 5.5, and the standard deviation is approximately 2.872.'), type='text')], created_at=1713712521, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_7uDcwWCXMQdvq5n00eCiqVwh', status=None, thread_id='thread_LYgCQpvVNdJFSPoTLG1GdLGW'), Message(id='msg_KnDBqmF5z3XuDM2X5kfC2X8N', assistant_id=None, completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Calculate both the mean and standard deviation of this series of numbers:\n              \n            [1,2,3,4,5,6,7,8,9,10]\n    '), type='text')], created_at=1713712514, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_LY